# How to Convert a Time Series to a Supervised Learning Problem in Python

[原网页](https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/)

机器学习方法如深度学习可被用于时序预测。

时许预测问题需要被重新设计为有监督学习问题后才能使用机器学习。从一个序列转为输入对和输出序列。

这个教程的内容：

- 如何编写函数来将一个时序数据集转换为有监督学习数据集。
- 如何转化单变量时序数据用于机器学习。
- 如何转化多变量时序数据用于机器学习。

## Time Series vs Supervised Learning

时序是一个通过time index来排序的数字序列。

一个有监督学习问题是由输入模式（X）和输出模式（Y）组成的，由此算法可以学习到如何由输入模式预测输出模式。

## Pandas shift() Function

给出一个DataFrame，shift()函数用于生成某一列向前移或向后移的副本（上下滚动）。

这个举措用于创建lag observation的列以及创建用于对在有监督学习格式下的时序数据集的forecast observation的列。

下面是一个简单的例子。

In [9]:
from pandas import DataFrame
df = DataFrame()
df['t'] = [x for x in range(10)]
print(df)

   t
0  0
1  1
2  2
3  3
4  4
5  5
6  6
7  7
8  8
9  9


可以把所有observation都向下滚一个time step。把序列向前滚一步可以构成一个原始的有监督学习问题，尽管X和y的顺序反了。

In [2]:
df = DataFrame()
df['t'] = [x for x in range(10)]
df['t-1'] = df['t'].shift(1)
print(df)

   t  t-1
0  0  NaN
1  1  0.0
2  2  1.0
3  3  2.0
4  4  3.0
5  5  4.0
6  6  5.0
7  7  6.0
8  8  7.0
9  9  8.0


shift函数的参数也可为负数，代表往上移。

In [1]:
df = DataFrame()
df['t'] = [x for x in range(10)]
df['t+1'] = df['t'].shift(-1)
print(df)

   t  t+1
0  0  1.0
1  1  2.0
2  2  3.0
3  3  4.0
4  4  5.0
5  5  6.0
6  6  7.0
7  7  8.0
8  8  9.0
9  9  NaN


运行代码之后可以将't'列作为输入X，'t+1'列作为输出y。输入值0可用于预测输出值1。

时许预测术语中，当前时间(t)和未来时间(t+1, t+n)是预测时间而过去观测时间(t-1, t-n)是用来做出预测的。

正移和负移可用于从具有有监督学习问题的输入和输出模式序列的时间序列创建新的DataFrame。这不仅可用于经典的X->y预测，还可以进行X->Y预测，X、Y均为序列。

shift函数亦可用于多参时许问题，在本tutorial后面会有讲解。

## series_to_supervised() 函数

函数参数包括：

- **data** ：作为list或者2D NumPy array的观测值序列。必需。
- **n_in** ：作为输入(X)的滞后观察值的数量。值在1到len(data)之间，可选，默认为1。
- **n_out** ：作为输出(y)的观测值的数量。值在0到len(data)-1之间，可选，默认为1。
- **dropnan** ：决定是否去除带NaN值的行的布尔值，可选，默认为True。

函数返回一个值：

- **return** ：用于有监督学习的Pandas DataFrame序列数据。

函数定义中包含默认的参数，可以直接传入数据调用，它会构建以t-1作为X、t作为y的DataFrame。

In [ ]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

dropna函数的文档：

![](images/2-img1.png)

## One-Step单变量预测

用t-1的数据来预测现在t刻的结果。

In [5]:
values = [x for x in range(10)]
data = series_to_supervised(values)
print(data)

   var1(t-1)  var1(t)
1        0.0        1
2        1.0        2
3        2.0        3
4        3.0        4
5        4.0        5
6        5.0        6
7        6.0        7
8        7.0        8
9        8.0        9


从运行结果可以看到观测值被命名为“var1”，输入观测值命名为（t-1），输出time step被命名为（t）。

带NaN值的行被自动移除出了DataFrame。

也可以通过传入任意长度的输入序列，如3。

> data = series_to_supervised(values, 3)

In [4]:
data = series_to_supervised(values, 3)
print(data)

   var1(t-3)  var1(t-2)  var1(t-1)  var1(t)
3        0.0        1.0        2.0        3
4        1.0        2.0        3.0        4
5        2.0        3.0        4.0        5
6        3.0        4.0        5.0        6
7        4.0        5.0        6.0        7
8        5.0        6.0        7.0        8
9        6.0        7.0        8.0        9


## Multi-Step或序列预测

另一种预测问题是用过去的观测值预测未来观测值的序列。

In [6]:
values = [x for x in range(10)]
data = series_to_supervised(values, 2, 2)
print(data)

   var1(t-2)  var1(t-1)  var1(t)  var1(t+1)
2        0.0        1.0        2        3.0
3        1.0        2.0        3        4.0
4        2.0        3.0        4        5.0
5        3.0        4.0        5        6.0
6        4.0        5.0        6        7.0
7        5.0        6.0        7        8.0
8        6.0        7.0        8        9.0


## 多变量预测

有多个变量观测值，需要预测其中一个或多个变量。

In [7]:
raw = DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]
values = raw.values
data = series_to_supervised(values)
print(data)

   var1(t-1)  var2(t-1)  var1(t)  var2(t)
1        0.0       50.0        1       51
2        1.0       51.0        2       52
3        2.0       52.0        3       53
4        3.0       53.0        4       54
5        4.0       54.0        5       55
6        5.0       55.0        6       56
7        6.0       56.0        7       57
8        7.0       57.0        8       58
9        8.0       58.0        9       59


具体问题具体分析，分类到X和Y的列可自行确认，如当前var1的观察值也可作为输入，只需预测var2。